### Import packages

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from functools import reduce

### Import all files from directories

In [2]:
#albcr = [f for f in listdir("albcr") if isfile(join("albcr", f))]
bio = [f for f in listdir("bio") if isfile(join("bio", f))]
cbc = [f for f in listdir("cbc") if isfile(join("cbc", f))]
chol = [f for f in listdir("chol") if isfile(join("chol", f))]
cholhdl = [f for f in listdir("cholhdl") if isfile(join("cholhdl", f))]
demo = [f for f in listdir("demo") if isfile(join("demo", f))]
glu = [f for f in listdir("glu") if isfile(join("glu", f))]
hdl = [f for f in listdir("hdl") if isfile(join("hdl", f))]
trildl = [f for f in listdir("trildl") if isfile(join("trildl", f))]

#### Give string values to gender

In [3]:
def gender(x):
    x = int(x)
    if x == 1:
        return "M"
    elif x == 2:
        return "F"

### Extracting demographical data

In [4]:
demo_data = pd.DataFrame()
for i in demo:
    temp = pd.read_sas("demo/"+i)
    temp = temp[["SEQN","RIAGENDR","RIDAGEYR"]]
    temp.columns = ["SEQN","GENDER","AGE"]
    temp = temp.astype({"AGE":int,"SEQN":int})
    temp["GENDER"] = temp["GENDER"].apply(gender)
    demo_data = pd.concat([demo_data,temp],axis=0)
    del temp

### Extracting Albumin and Creatinine 

In [5]:
# albcr_data = pd.DataFrame()
# for i in albcr:
#     temp = pd.read_sas("albcr/"+i)
#     if "URXUMS" in temp:
#         albumin = "URXUMS"
#     elif "URXUMASI" in temp:
#         albumin = "URXUMASI"
#     temp = temp[["SEQN",albumin,"URXUCR"]]
#     temp.columns = ["SEQN","Albumin_mgl","Creatinine_mgdl"]
#     temp = temp.astype({"SEQN":int})
#     albcr_data = pd.concat([albcr_data,temp],axis=0)
#     del temp

### Extracting data from standard biochemistry profile

In [5]:
bio_data = pd.DataFrame()
for i in bio:
    temp = pd.read_sas("bio/"+i)
    if "LBXSAPSI" in temp:
        alkPho = "LBXSAPSI"
    elif "LBDSAPSI" in temp:
        alkPho = "LBDSAPSI"
    temp = temp[["SEQN","LBXSKSI","LBXSNASI","LBDSTBSI","LBDSCASI","LBXSTP","LBDSBUSI",alkPho,"LBDSALSI","LBDSCRSI"]]
    temp.columns = ["SEQN","Potassium_mmolL","Sodium_mmolL","Bilirubin_umolL","Calcium_mmolL","Protein_gdL","Urea_mmolL",
                   "AlkPhos_UL","Albumin_gL","Creatinine_umolL"]
    temp = temp.astype({"SEQN":int})
    bio_data = pd.concat([bio_data,temp],axis=0)
    del temp

### Extracting Glucose

In [6]:
glu_data = pd.DataFrame()
for i in glu:
    temp = pd.read_sas("glu/"+i)
    if "LBDGLUSI" in temp:
        glucose = "LBDGLUSI"
    elif "LBXGLUSI" in temp:
        glucose = "LBXGLUSI"
    else:
        print(i)
    temp = temp[["SEQN",glucose]]
    temp.columns = ["SEQN","Glucose_mmolL"]
    temp = temp.astype({"SEQN":int})
    glu_data = pd.concat([glu_data,temp],axis=0)
    del temp

### Extracting LDL and Triglyceride

In [7]:
trildl_data = pd.DataFrame()
for i in trildl:
    temp = pd.read_sas("trildl/"+i)
    temp = temp[["SEQN","LBDLDLSI","LBDTRSI"]]
    temp.columns = ["SEQN","LDL_mmolL","Triglyceride_mmolL"]
    temp = temp.astype({"SEQN":int})
    trildl_data = pd.concat([trildl_data,temp],axis=0)
    del temp

### Extracting HDL

In [8]:
hdl_data = pd.DataFrame()
for i in hdl:
    temp = pd.read_sas("hdl/"+i)
    temp = temp[["SEQN","LBDHDDSI"]]
    temp.columns = ["SEQN","HDL_mmolL"]
    temp = temp.astype({"SEQN":int})
    hdl_data = pd.concat([hdl_data,temp],axis=0)
    del temp

### Extracting Total Cholesterol

In [9]:
chol_data = pd.DataFrame()
for i in chol:
    temp = pd.read_sas("chol/"+i)
    temp = temp[["SEQN","LBDTCSI"]]
    temp.columns = ["SEQN","Cholesterol_mmolL"]
    temp = temp.astype({"SEQN":int})
    chol_data = pd.concat([chol_data,temp],axis=0)
    del temp

#### Merge HDL and Cholesterol

In [10]:
chol_hdl = pd.merge(hdl_data,chol_data,on=["SEQN"],how="outer")

### Extracting HDL and Cholesterol

In [11]:
x=0
cholhdl_data = pd.DataFrame()
for i in cholhdl:
    temp = pd.read_sas("cholhdl/"+i)
    if "LBDHDDSI" in temp:
        hdl = "LBDHDDSI"
    elif "LBDHDLSI" in temp:
        hdl = "LBDHDLSI"
    temp = temp[["SEQN",hdl,"LBDTCSI"]]
    temp.columns = ["SEQN","HDL_mmolL","Cholesterol_mmolL"]
    temp = temp.astype({"SEQN":int})
    cholhdl_data = pd.concat([cholhdl_data,temp],axis=0)
    del temp

#### Concat both

In [12]:
cholhdl_data = pd.concat([cholhdl_data,chol_hdl],axis=0)

### Extracting Complete blood count

In [13]:
x=0
cbc_data = pd.DataFrame()
for i in cbc:
    temp = pd.read_sas("cbc/"+i)
    temp = temp[["SEQN","LBXRBCSI","LBXHGB","LBXHCT","LBXMCVSI","LBXMC","LBXPLTSI"]]
    temp.columns = ["SEQN","RBC_MuL","Hemoglobin_gdl","Hematocrit_%","MCV_fL","MCHC_gdl","Platelet_TuL"]
    temp = temp.astype({"SEQN":int})
    cbc_data = pd.concat([cbc_data,temp],axis=0)
    del temp

### Merge Dataframes

In [15]:
dfs = [demo_data,glu_data,bio_data,trildl_data,cholhdl_data,cbc_data]
data = reduce(lambda  left,right: pd.merge(left,right,on=['SEQN'],
                                            how='outer'), dfs)

### Drop NaN values

In [16]:
data = data.dropna()

### For age 20+

In [17]:
data = data[data["AGE"]>=20]

### Reorder Columns

In [18]:
data = data[["SEQN","GENDER","Albumin_gL","Glucose_mmolL","Urea_mmolL","Cholesterol_mmolL","Protein_gdL","Sodium_mmolL",
             "Creatinine_umolL","Hemoglobin_gdl","Bilirubin_umolL","Triglyceride_mmolL","HDL_mmolL","LDL_mmolL",
             "Calcium_mmolL","Potassium_mmolL","AlkPhos_UL","Hematocrit_%","MCHC_gdl","MCV_fL","Platelet_TuL","RBC_MuL","AGE"]]

In [19]:
data.head(10)

,SEQN,GENDER,Albumin_gL,Glucose_mmolL,Urea_mmolL,Cholesterol_mmolL,Protein_gdL,Sodium_mmolL,Creatinine_umolL,Hemoglobin_gdl,...,LDL_mmolL,Calcium_mmolL,Potassium_mmolL,AlkPhos_UL,Hematocrit_%,MCHC_gdl,MCV_fL,Platelet_TuL,RBC_MuL,AGE
1,2,M,45.0,4.646,6.8,5.56,7.2,144.1,61.9,14.1,...,3.52,2.325,4.06,62.0,41.8,33.6,88.5,214.0,4.73,77
4,5,M,45.0,5.550,5.7,7.21,7.3,137.5,70.7,14.5,...,4.34,2.375,4.63,63.0,43.6,33.3,84.9,209.0,5.13,49
6,7,F,45.0,4.756,3.6,6.34,8.1,143.2,53.0,13.4,...,3.28,2.450,4.25,75.0,40.2,33.3,87.4,244.0,4.60,59
9,10,M,42.0,4.989,4.6,3.62,7.6,140.9,79.6,15.4,...,2.07,2.350,4.28,86.0,46.2,33.5,92.3,167.0,5.00,43
11,12,M,47.0,4.606,7.1,4.03,7.2,141.3,88.4,16.0,...,2.30,2.200,3.81,63.0,48.1,33.3,83.5,357.0,5.76,37
14,15,F,44.0,5.484,4.3,5.15,7.2,135.7,53.0,13.3,...,3.39,2.300,4.19,31.0,40.2,33.1,97.1,255.0,4.14,38
19,20,F,44.0,5.033,3.6,3.75,7.5,136.3,26.5,14.5,...,2.04,2.225,4.07,50.0,41.2,35.1,91.0,220.0,4.53,23
24,25,F,43.0,5.700,5.0,4.11,7.4,136.3,44.2,13.4,...,1.66,2.200,4.37,53.0,38.3,35.0,88.7,337.0,4.31,42
28,29,M,46.0,7.894,11.8,5.59,7.8,141.5,79.6,15.8,...,3.44,2.425,4.12,70.0,44.7,35.3,93.1,232.0,4.81,62
33,34,F,44.0,5.339,3.6,5.04,7.5,143.1,35.4,12.4,...,3.49,2.475,4.14,65.0,37.8,32.8,87.3,373.0,4.32,38


### Export to CSV

In [20]:
data.to_csv(r"D:\A_SJSU\Python\Program\Jupyter_notebook\ML\aging clock\data\data_bio_albcr.csv",index=False,header=True)

### Data for Deep Learning

In [21]:
# Removing SEQN and assigning binary for gender
data["GENDER"] = data["GENDER"].map(dict(M=0, F=1))
data = data.drop(["SEQN"],axis=1)

In [22]:
data.to_csv(r"D:\A_SJSU\Python\Program\Jupyter_notebook\ML\aging clock\data\data_bio_albcr_DL.csv",index=False,header=True)